In [3]:
#  get files
import pandas as pd

df = pd.read_csv(r"C:\Users\christine\Desktop\datamining\selected_scam_images.csv") # CHANGE HERE

img_folder = r"C:\Users\christine\Desktop\datamining\data\scamimage" + "\x5c"  #CHANGE HERE

df.shape

# df = df.iloc[0:500]
df = df.iloc[500:1000]
df

,images
500,Yafe661.jpg
501,Yafe662.jpg
502,blueray011.jpg
503,Richy20301.jpg
504,Richy20302.jpg
...,...
995,john19642.jpg
996,pagej1.jpg
997,pagej2.jpg
998,pagej3.jpg


In [5]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.applications.xception import Xception
from keras.models import load_model
from pickle import load
import numpy as np
from PIL import Image
from glob import glob
import json

def extract_features(filename, model):
        try:
            image = Image.open(filename)
            
        except:
            print("ERROR: Couldn't open image! Make sure the image path and extension is correct")
        image = image.resize((299,299))
        image = np.array(image)
        # for images that has 4 channels, we convert them into 3 channels
        if image.shape[2] == 4: 
            image = image[..., :3]
        image = np.expand_dims(image, axis=0)
        image = image/127.5
        image = image - 1.0
        feature = model.predict(image)
        return feature

def word_for_id(integer, tokenizer):
 for word, index in tokenizer.word_index.items():
     if index == integer:
         return word
 return None


def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo,sequence], verbose=0)
        pred = np.argmax(pred)
        word = word_for_id(pred, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text

max_length = 32 
tokenizer = load(open("models/tokenizer.p","rb"))
model = load_model('models/model_9.h5')
xception_model = Xception(include_top=False, pooling="avg")
labels = {}

problematic = []

for i in df.images:
    try: 
        path = (img_folder +i)
        photo = extract_features(path, xception_model)
        img = Image.open(path)
        description = generate_desc(model, tokenizer, photo, max_length)
        description = description[6:-4]
        labels[i] = description
        print(path)

    except:
        problematic.append(i)
    

json_object = json.dumps(labels) 

# change the filename to your name
with open('scam_500_1000.json', 'w') as f:
  json.dump(labels, f, ensure_ascii=False)

print('\n\n DONE ALR :DDD')

1/1 [==============================] - 3s 3s/step
C:\Users\christine\Desktop\datamining\data\scamimage\Yafe661.jpg
1/1 [==============================] - 0s 161ms/step
C:\Users\christine\Desktop\datamining\data\scamimage\Yafe662.jpg
1/1 [==============================] - 0s 168ms/step
C:\Users\christine\Desktop\datamining\data\scamimage\blueray011.jpg
1/1 [==============================] - 0s 166ms/step
C:\Users\christine\Desktop\datamining\data\scamimage\Richy20301.jpg
1/1 [==============================] - 0s 160ms/step
C:\Users\christine\Desktop\datamining\data\scamimage\Richy20302.jpg
1/1 [==============================] - 0s 170ms/step
C:\Users\christine\Desktop\datamining\data\scamimage\Richy20303.jpg
1/1 [==============================] - 0s 157ms/step
C:\Users\christine\Desktop\datamining\data\scamimage\Richy20304.jpg
1/1 [==============================] - 0s 156ms/step
C:\Users\christine\Desktop\datamining\data\scamimage\Richy20305.jpg
1/1 [==============================] - 0s

In [6]:
problematic

[]